<a href="https://colab.research.google.com/github/sebasesto/CEIA/blob/master/NLP/Sesto_3b_Custom_embedding_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['bjork.txt',
 'janisjoplin.txt',
 'lil-wayne.txt',
 'missy-elliott.txt',
 'lin-manuel-miranda.txt',
 'bruce-springsteen.txt',
 'johnny-cash.txt',
 'leonard-cohen.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'blink-182.txt',
 'lady-gaga.txt',
 'nicki-minaj.txt',
 'disney.txt',
 'bieber.txt',
 'prince.txt',
 'bob-marley.txt',
 'kanye-west.txt',
 'ludacris.txt',
 'patti-smith.txt',
 'dolly-parton.txt',
 'notorious_big.txt',
 'alicia-keys.txt',
 'amy-winehouse.txt',
 'dr-seuss.txt',
 'drake.txt',
 'radiohead.txt',
 'eminem.txt',
 'britney-spears.txt',
 'notorious-big.txt',
 'Lil_Wayne.txt',
 'kanye.txt',
 'paul-simon.txt',
 'adele.txt',
 'dj-khaled.txt',
 'nirvana.txt',
 'jimi-hendrix.txt',
 'nickelback.txt',
 'al-green.txt',
 'r-kelly.txt',
 'bob-dylan.txt',
 'cake.txt',
 'dickinson.txt',
 'beatles.txt',
 'michael-jackson.txt',
 'nursery_rhymes.txt',
 'lorde.txt',
 'Kanye_West.txt',
 'joni-mitchell.txt']

In [4]:
# !cat songs_dataset/beatles.txt
!cat songs_dataset/britney-spears.txt | wc -l

3847


In [5]:
# Selección de banda y armado del dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/britney-spears.txt', sep='/n', header=None, engine='python')
df.head(10)

,0
0,They say get ready for the revolution
1,I think it's time we find some sorta solution
2,Somebody's caught up in the endless pollution
3,"They need to wake up, stop living illusions I ..."
4,Why won't somebody feel this
5,This is my wish that we all feel connected
6,This is my wish that nobodies neglected Be lik...
7,Be like a rocket Take off
8,"Just fly, away (ay, ay)"
9,To find your space Take off


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3848


### 1 - Preprocesamiento

**Tokenización**

In [7]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Demos un vistazo
sentence_tokens[:3]

[['they', 'say', 'get', 'ready', 'for', 'the', 'revolution'],
 ['i', 'think', "it's", 'time', 'we', 'find', 'some', 'sorta', 'solution'],
 ["somebody's", 'caught', 'up', 'in', 'the', 'endless', 'pollution']]

### 2 - Crear los vectores (word2vec)

In [9]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

**Creamos el modelo generador de vectoeres**


In [10]:
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=3,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=400,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de docs en el corpus: 3848
Cantidad de words distintas en el corpus: 876


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 229064.71875
Loss after epoch 1: 143827.03125
Loss after epoch 2: 141329.65625
Loss after epoch 3: 141808.96875
Loss after epoch 4: 139713.3125
Loss after epoch 5: 134291.6875
Loss after epoch 6: 127147.5
Loss after epoch 7: 111784.375
Loss after epoch 8: 109088.0
Loss after epoch 9: 103402.625
Loss after epoch 10: 99643.375
Loss after epoch 11: 96536.625
Loss after epoch 12: 93284.125
Loss after epoch 13: 91902.5
Loss after epoch 14: 89015.5
Loss after epoch 15: 85550.625
Loss after epoch 16: 86388.75
Loss after epoch 17: 83217.125
Loss after epoch 18: 77814.0
Loss after epoch 19: 76658.75
Loss after epoch 20: 75684.75
Loss after epoch 21: 74920.5
Loss after epoch 22: 74307.25
Loss after epoch 23: 73994.25
Loss after epoch 24: 73362.0
Loss after epoch 25: 72535.0
Loss after epoch 26: 72065.5
Loss after epoch 27: 72415.5
Loss after epoch 28: 72016.0
Loss after epoch 29: 72260.25
Loss after epoch 30: 70878.25
Loss after epoch 31: 71424.25
Loss after epoch 32: 70788.0

(3422426, 5614200)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["heart"], topn=10)

[('loud', 0.6058056354522705),
 ('beats', 0.6007773876190186),
 ('played', 0.5649458169937134),
 ('broken', 0.5470932722091675),
 ('supreme', 0.5341468453407288),
 ('within', 0.5201370716094971),
 ('lonely', 0.4943247437477112),
 ('pleasure', 0.4828777313232422),
 ("heart's", 0.4815444350242615),
 ('cries', 0.4761291444301605)]

In [24]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["again"], topn=10)

[('the', 0.008150998502969742),
 ('ahhhh', -0.03777128458023071),
 ('babe', -0.04839927703142166),
 ('should', -0.05040828883647919),
 ('gonna', -0.05301067233085632),
 ('physical', -0.05491059273481369),
 ('floor', -0.06445908546447754),
 ('change', -0.0656035915017128),
 ('down', -0.06786665320396423),
 ('taste', -0.07432958483695984)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('haha', 0.5697326064109802),
 ('criminal', 0.47140106558799744),
 ('wings', 0.4637596607208252),
 ('owh', 0.44797828793525696),
 ('surprise', 0.442110151052475),
 ('ohhhh', 0.42615771293640137),
 ('wo', 0.41629523038864136),
 ('pink', 0.41216176748275757),
 ('use', 0.41123485565185547),
 ('hate', 0.3970568776130676),
 ('being', 0.394145667552948),
 ('curiosity', 0.39407360553741455),
 ('begging', 0.38764744997024536),
 ('simply', 0.3855835199356079),
 ('town', 0.3838420510292053)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["happy"], topn=5)

[('born', 0.6264002323150635),
 ('dreaming', 0.5275756120681763),
 ('tryna', 0.4986525774002075),
 ('remember', 0.4666686952114105),
 ('wo', 0.4631897807121277)]

### 5 - Visualizar agrupación de vectores

In [18]:
def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

## Conclusiones


En la sección 4 de ensayar se pueden notar la similitud o cercanía entre algunas de las palabras en el espacio de embeddings.
Por ejemplo la palabra **heart** tiene gran cercanía con **broken**, esto puuede deberse a la canción "From the bottom of my broken heart", tambien su similitud con **heart's**. Tambien de la canción "Lucky" se puede relacionar **lonely** y **cries** por la frase: Cries in her lonely heart.

La palabra **again** se puede estar relacionando con **babe** por la canción I did it again en las frases: Oh baby, baby Oops, I did it again. Y tambien con **should** por: you shouldn't have Oops, I did it again to your heart

La palabra **love** en la frase But mama I'm in love with a criminal se puede asociar con **criminal**

De la palabra **happy** se puede justificar la similitud con **born* por la canción "Born to make you happy"